In [403]:
import os,sys
from Bio import SeqIO, Align
import pandas as pd
import re

base_dir = '/home/john.palmer/work/norovirus/requests/pairwise_pipeline_sanger/'
os.chdir(base_dir)

def init_aligner(mode='global'):
	aligner = Align.PairwiseAligner()
	aligner.mode = mode
	aligner.open_gap_score = -10
	aligner.extend_gap_score = -0.1
	aligner.target_end_gap_score = 0.0
	aligner.query_end_gap_score = 0.0
	
	return aligner

def get_boundaries (str):
	gap_prefix = re.compile('^[-]+')
	gap_suffix = re.compile('[-]+$')
	# return a tuple giving indices of subsequence without gap prefix and suffix
	res = [0,len(str)]
	left = gap_prefix.findall(str)
	right = gap_suffix.findall(str)
	if left:
		res[0] = len(left[0])
	if right:
		res[1] = len(str) - len(right[0])
	return res

def pairwise_align(aligner, ref, query, cut=False):
	# Perform pairwise alignment and get aligned sequences 
	align_iter = aligner.align(ref.seq, query.seq)
	ref_aligned, qry_aligned = next(align_iter)

	return ref_aligned, qry_aligned


In [396]:
# contig_dict = {
# 	'FV17557': [0], 
# 	#'FV19890': [1,2,7],
#  	#'FV22276': [0,1,17,30], 
# 	'FV22289': [0],
# 	'FV22141': [0],
# 	'FV2262': [0],
# }

# aligner = init_aligner()

# for name in contig_dict:
# 	print(name)
# 	contig_seqs = list(SeqIO.parse('contigs/'+name + '.contigs.fa', 'fasta'))
# 	contig_seqs = [x[4000:] for x in contig_seqs]
# 	sanger = next(SeqIO.parse('sanger/'+name + '.fasta', 'fasta'))


# 	for idx in contig_dict[name]:
# 		ref, qry = pairwise_align(aligner, contig_seqs[idx], sanger)
# 		print(contig_seqs[idx].id)
# 		print(ref)
# 		print("SANGER:")
# 		print(qry)
# 		print(get_boundaries(qry))


In [411]:
contig_dict = {
	'FV17557': ['AB045603|GII.12|Gifu96','MK764013|GII.4-Sydney-Wayne0078|Wayne0078'], 
	#'FV19890': ['MK764013|GII.4-Sydney-Wayne0078|Wayne0078','KJ196299|GII.12|SaitamaT18'],
	'FV22141': ['JX459908|GII.4-Sydney|NSW0514','AY134748|GII.2|SnowMountain'],
 	#'FV22276': ['AY134748|GII.2|SnowMountain','AB039782|GII.3|SaitamaU201'], 
	'FV22289': ['MK764013|GII.4-Sydney-Wayne0078|Wayne0078','AY134748|GII.2|SnowMountain'],
	'FV2262': ['AY134748|GII.2|SnowMountain','OP811331|GII.15|NIH-PAK-001'],
}

contig_dict = {
	'FV22141': ['GU445325|GII.P4|NewOrleans','KY865306|GII.P16|SantaRosa1764'],
	'FV2262': ['KY865306|GII.P16|SantaRosa1764','KJ196290|GII.P15|SapporoHK299'],
}

aligner = init_aligner(mode='global')

ref_dict = SeqIO.to_dict(SeqIO.parse("/home/john.palmer/git/noro-meta-nf/cache/blast_db/gene/polymerase/ptype_blastdb.fasta",'fasta'))

for name, ref_names in contig_dict.items():
	print(name)

	contig_seq = next(SeqIO.parse('contigs/'+name + '.contigs.fa', 'fasta'))
	sanger_seq = next(SeqIO.parse('sanger/'+name + '.fasta', 'fasta'))


	c_ref, c_qry = pairwise_align(aligner, ref_dict[ref_names[0]], contig_seq)
	s_ref, s_qry = pairwise_align(aligner, ref_dict[ref_names[1]], sanger_seq)

	print("CONTIG ALIGNMENT:")
	print(c_ref)
	print(c_qry)

	print("SANGER ALIGNMENT:")
	print(s_qry)
	print(s_ref)

FV22141
CONTIG ALIGNMENT:
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
2262
CONTIG (pipeline):
qseqid	sseqid	pident	alignment_length	bitscore
NODE_1_length_7621_cov_12741.301841     KY865306|GII.P16|SantaRosa1764  97.839  1527    2638
qry: GACGCCATCTTCATTCACGAAATTGGGAGCCAGATTGCGATCGCCCTCCCACGTGCTCAAGTCAGAAAACCTCATCCACCTGAACATGGGTTCCTGCCTTGGCACGTAAAAGTCCATTCCACCTTCTTTGAGTTCAGTTATGACTAATTTACTGATTTTACTGTAGAAAGAGGGTCCGTGAAGTGAGGCTTCACCAAGCAGTGCCATAAGCTGTATGGGTCTTTGAGAGTGGGGTATCATTGTCTCATCGGGGTCTTCATGATTTGGTCCTCTAGTCCAGTACATCTGCCTCAAAATTGAGCTTTGGTCCAGTTTTCCAAACCAGCCAGCTGGGTCACGAGTCACCGTCCTCCGGAGAAAAGTGAGACCGTTCAAATCTTCACTGATAATCAGGGGTCCCTCGGTTTTGTCTGGGCGGGTTGGTTTTAGGCCGTACTCCTTCAATTTGGCGGTTAACTGTTCAGGGTCCAATTTTATGTCGGTGCTGACAATCTCGTCATCACCGTAAAATGAGAACATGGAGTTTGCCTGTATAATGTCAGGGGACAGTTTGGTGACTTCAGACAAGGCACAGAGGGTTAGCAGCCAGTGTGCGATGGAGTTCCATTGAGAAGTGCATGGTACACCAGAAGGGAGCCCTTCATTTATTGTAATTTTAAAGTCTCCTACATCTACTACACTAGGGGCTAGCAGATCCTCAGCGACTATTTGTGCCAATTGTGGTTCTGCAGAGAATCTGACCATGATTTCCAAGGCTGCTGCTAGTACTGCCCTCTGTTGTGTTGAATCCCAACGAGAGTAGTCTGCATCATAGTGATATTTATATCTGGAATGTTTCTCAAATATTATTGGGCCATCTTCATTCATATTCATGCCAACTCGGACTGGGAGTGATATGCAGTGTGCCTTCATCTCGTCCATAAGCCCACCAAATGACCTAGCGCACCGGATCATGGTGGACAAGTCAGAGCCCCAGAGCAATCTCTTCTTGATCTTTCCATAGATTTTCTCAGTCTTGACTAGCTCGTCCTTGAGTGCTGCTGTGTACACTGGTGTCATGTGTTTCCCTTCCTCAAACATTAGGTTTGCTTTTGATGCTTGGTCTGCCAGTTTACCAGTGAAGGTCTCACCATTCCAGAATTCATTCTTTCGGGCGTGATGGGGGTGTCCGCTGGAGGTGGTTTTGTCAAGTGAGGCACACGCTTGTGCGTATGACCATTTTTGTGGAGGGTCCAGGGTTTGTTCGAGGACATTGATGATGGTTTGTTTGGCTGCTTCCAATACACTTGGTCTTGGAGGTTTGCCCCTGGGTTCAGTGAATGGCTTCAATTGGTCTCTCATTACCTGCTGCAGTGAAGGCCCACCCTTAACACGTGGGTCACGGCCACCAAGGTAGGCAGGCTCATATGTCCCTGGTGGAAGGGGCGTGTTCGATGATCTCCAGAATTTGGTTTTGGTGCTCAATTTTGGTGCACCCCCAGGGCCTAGAATGGGTGCCCCACAGTATGTTCCTTTGTCATCTCCACC
ref: GACGCCATCTTCATTCACAAAATTGGGAGCCAGATTGCGATCGCCCTCCCACGTGCTCAAGTCAGAAAACCTCATCCACCTGAACATGGGTTCCTGCCTTGGCACGTAAAAGTCCATCCCACCTTCTTTGAGTTCAGTTATGACCAATTTACTGATTTTACTGTAGAAAGAGGGTCCGTGAAGAGAGGCTTCACCAAGCAGTGCCATGAGCTGTATGGGTCTTTGGGAATGGGGTATCATTGTCTCATTGGGGTCTTCATGATTTGGTCCTCTAGTCCAGTACATCTGCCTCAAAATTGAGCTTTGGTCCAGTTTTCCAAACCAGCCAGCTGGGTCACGAGTCACCGTCCTTCGGAGGAAAGTGAGACCGTTCAAATCTTCACTGATAATCAGGGGTCCCTCGGTTTTGTCTGGGCGGGTTGGTTTCAGGCCGTACTCCTTCAATTTGGCGGTTAACTGTTCAGGGTCCAATTTTATGTCGGTACTGACAATCTCGTCATCACCGTAAAATGAGAACATGGAATTTGCCTGTATAATGTCAGGGGACAGTTTGGTGACTTCAGACAAGGCACAGAGAGTTAGCAGCCAGTGTGCGATGGAGTTCCATTGAGAAGTGCATGGCACACCAGAAGGGAGCCCTTCATTTATTGTAATTTTGAAGTCTCCTACATCTACTACACTAGGGGCCAGCAGATCCTCAGCGACTATTTGTGCCAATTGTGGTTCTGCAGAGAATCTGACCATGATTTCCAAGGCTGCTGCTAGAACTGCCCTCTGTTGTGTTGAATCCCAACGAGAGTAGTCTGCATCATAGTGATATTTGTATCTGGAATGTTTCTCAAATATTATTGGGCCATCTTCATTCATATTCATGCCAACTCGGACTGGGAGTGATATGCAGTGTGCCTTCATCTCGTCCATAAGCCCACCAAATGACCTAGCGCACCGGATCATGGTGGACAAGTCAGAGCCCCAGAGCAGTCTCTTCTTGATCTTTCCATAGATTTTCTCAGTCTTGACTAGCTCGTCCTTGAGTGCTGCTGTGTACACTGGTGTCATGTGTTTCCCTTCCTCAAACATTAGGTTTGCTTTTGATGCTTGGTCTGCCAGTTTACCAGTGAAGGTCTCACCATTCCAGAATTCATTCTTTCGGACGTGATGGGGGTGCCCGCTGGAGGTGGTTTTGTCAAGTGAGGCACAAGCCTGTGCGTATGTCCATTTTTGTGGAGGGTCCAGGGTTTGTTCGAGGACATTGATGATGGTTTGTTTGGCTGCTTCCAATACACTTGGTCTTGGAGGTTTGCCCCTGGGTTCAGTGAATGGCTTCAACTGGTCTCTCATCACCTGCTGCAGGGAAGGCCCACCCTTAACACGCGGATCACGGCCACCGAGGTAGGCAGGCTCATATGTCCCTGGTGGAAGGGGCGTGTTCGATGACCTCCAAAATTTGGTTTTGGTGCTCAATTTTGGTGCACCCCCAGGGCCTAGAATGGGTGCCCCACAGTATGTTCCTTTGTCATCTCCACC

SANGER:
qseqid	sseqid	pident	alignment_length	bitscore
FV2262  KJ196290|GII.P15|SapporoHK299   92.664  259     374 
qry: --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------TGAAAACCCCCACGAGTGCATGGTCCCACACTCACAGCGCGCAACTCAGTTAATGGCGCTGTTGGGTGAGGCATCATTGCACGGTCCCCAGTTCTACAAGAAAGTCAGCAAGATGGTTATCAATGAGATCAAGAGTGGTGGTCTGGAATTTTACGTGCCCAGACAAGAGGCCATGTTCAGATGGATGAGATTCTCTGACCTCAGCACGTGGGAGGGCGATCGCAATCTGGCTCCCGACGGTGTGAATGAGGATGGCGTCGAGTGACGCGCCCGTTTCTGGCACCGATGGTGCGGCAGGACTCGTCCCAGAGAGTCAACAAGAAGTCTTGCCTTTAGAGCCCGTCGCGGGAGTTCAGCTGGCTGCTCCCGTAGCAGGGCAAAGTAATATAATTGACCCCTGGATTAGAATGAATTTTGTGCAAGCCCCGGCCGGTGAGTTCACTGTTTCCCCTCGGAACGCACCGGGTGAGTGCTTATTGATTTAGAATTAGGACCAGAATTAAATCCCTATCTTATCATCTTGCAGA
ref: GGAGGAGAGGACAAAGGAACATACTGTGGTGCCCCAATACTTGGACTTGGGAAGGCCCCTAAGCTCTGCACTAAAACAAAGTTCTGGCGTTCCTCACCTGATGCACTACCCCCGGGAACGTATGAACCAGCCTATCTAGGTGGAAAAGACCCAAGAGTTGAGAAGGGCCCATCACTCCAGCAGGTTATGAGAGAACAACTTAAACCCTTCACCGAACCCAGAGGAAAACCGCCAAAGCCCAGCGTTCTCGATGAGGCAAAGAAAACTGTGATGAATGTGTTGGAGCAAACCATAAACCCTGCCAAACCCTGGTCATACTCACAAGCATGTGCCTCACTTGACAAAACGACTTCAAGTGGTAGCCCACACCACCTAAGGAAGAATGATCACTGGAATGGTGAGTCTTTCACCGGCCCTTTAGCCGACCAAGCTTCAAAGGCAAACTTAATGTATGAACAGGCAAAACACGTGAGCCCCGTTTACACCGCAGCCCTCAAAGATGAACTAGTCAAGACTGACAAGATTTATAATAAGATAAAGAAGAGGTTGCTATGGGGTTCTGACCTGGGTACCATGGTCAGGTGTGCCAGAGCCTTTGGTGGGTTAATGGACAGCATGAAAGAGAGTTGTGTGATGTTGCCCTGCAGAGTTGGAATGAATATGAATGAGGATGGGCCAATCATTTTTGACAAACATGCAAAATATAAATACCACTATGATGCAGACTACTCCAGATGGGACTCCACGCAGCAAAGATGCATCTTGTCTGCTGCCATGGAGGTTATGGTGAAATTCTCAGCTGAACCGGAATTGGCACAGGTGGTGGCTGAAGACCTACTCGCCCCGAGCCAACTCGACGTGGGCGATTTTGTGGTGTCAGTTCAGGAAGGCCTTCCTTCAGGTGTCCCATGTACCTCCCAGTGGAACTCTATAGCACACTGGATCATTACCCTTAGTGCAATGTCAGAAGTCTCTGGTCTCTCCCCAGATGTGATCCAAGCCCACTCATGCTTCTCTTTTTATGGTGATGATGAGATAGTTAGCACAGATATTAATCTGGACCCAGCAAAATTAACCTTGAAACTTAGGGAATATGGATTGGTCCCCACCAGACCAGACAAAACTGAGGGCCCACTAGTCATTACAGAGAACCTCCACGGGTTGACCTTCCTACGCAGGCACATCACACGAGACCCTGCTGGATGGTTTGGTAAGTTAGACCAGGATTCAATTTTGAGACAGCTCTATTGGACCAGAGGACCCAACCATGAGAATCCCTATGAGAGCATGGTTCCGCACTCACAGCGCGCGACCCAGCTAATGGCGTTGTTGGGTGAGGCATCGCTACACGGTCCCCAGTTTTACAAGAAAGTTAGCAAGATGGTCATCAATGAGATCAAAAGTGGTGGTCTGGAATTTTACGTGCCCAGACAGGAGGCCATGTTCAGATGGATGAGATTCTCTGACCTCAGCACGTGGGAGGGCGATCGCAATCTGGCTCCCGACGGTGTGAATGAGGATGGCGTC----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [413]:
contig_dict = {
	'FV17557': ['AB045603|GII.12|Gifu96','MK764013|GII.4-Sydney-Wayne0078|Wayne0078'], 
	#'FV19890': ['MK764013|GII.4-Sydney-Wayne0078|Wayne0078','KJ196299|GII.12|SaitamaT18'],
	'FV22141': ['JX459908|GII.4-Sydney|NSW0514','AY134748|GII.2|SnowMountain'],
 	#'FV22276': ['AY134748|GII.2|SnowMountain','AB039782|GII.3|SaitamaU201'], 
	'FV22289': ['MK764013|GII.4-Sydney-Wayne0078|Wayne0078','AY134748|GII.2|SnowMountain'],
	'FV2262': ['AY134748|GII.2|SnowMountain','OP811331|GII.15|NIH-PAK-001'],
}

# contig_dict = {
# 	'FV22141': ['GU445325|GII.P4|NewOrleans','KY865306|GII.P16|SantaRosa1764'],
# 	'FV2262': ['KY865306|GII.P16|SantaRosa1764','KJ196290|GII.P15|SapporoHK299'],
# }

for name, seqs in contig_dict.items():
    print(name)
    print(seqs[0])
    print(seqs[1])
    c_ref, c_qry = pairwise_align(aligner, ref_dict[seqs[0]], ref_dict[seqs[1]])
    lines = ''
    for char1, char2 in zip(c_ref, c_qry):
        lines += ' ' if char1 == char2 else '|'
    print(c_ref)
    print(lines)
    print(c_qry)

FV22141
GU445325|GII.P4|NewOrleans
KY865306|GII.P16|SantaRosa1764
GGTGGTGACAACAAGGGGACATACTGTGGTGCACCAATCCTAGGCCCAGGGAGTGCCCCAACACTTAGCACCAAGACCAAATTCTGGAGATCGTCCACAGCATCACTCCCACCTGGCACCTATGAACCAGCCTATCTTGGTGGCAAGGACCCTAGAGTCAAGGGTGGCCCTTCACTGCAGCAAGTCATGAGGGAACAGTTGAAGCCATTCACAGAGCCAAGGGGCAAGCCACCAAAACCAAGTGTATTAGAAGCTGCCAAGAAGACCATCATCAATGTCCTTGAGCAAACAATTGATCCACCTGAGAAATGGTCGTTCGCACAAGCTTGCGCGTCCCTTGACAAGACCACTTCCAGTGGTCATCCGCACCACATGCGGAAAAACGACTGCTGGAACGGGGAGTCCTTCACAGGCAAGCTGGCAGACCAGGCTTCCAAGGCCAACCTGATGTTTGAAGAAGGGAAGAACATGACCCCAGTCTACACAGCTGCGCTCAAGGATGAGTTAGTTAAAACTGACAAAATTTATGGTAAGATCAAGAAGAGGCTTCTCTGGGGCTCGGACTTGGCGACCATGATCCGGTGTGCTCGGGCATTCGGAGGCCTAATGGATGAACTCAAAGCACACTGTGTCACACTTCCCATTAGAGTTGGCATGAATATGAATGAGGATGGCCCCATCATCTTCGAGAGGCATTCCAGGTACACATATCACTATGATGCTGATTACTCTCGATGGGATTCAACACAACAGAGAGCCGTGTTGGCAGCAGCTCTGGAAATCATGGTTAAATTCTCCCCAGAACCACATTTGGCTCAGGTAGTCGCGGAAGACCTTCTTTCTCCTAGCGTGGTGGACGTGGGCGACTTCACAATATCAATCAACGAGGGTCTTCCCTCTGGGGTGCCCTGTACCTCCCAATGGAACTCCATCG